In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

In [2]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime

#Reading Data from CSV Files
temperature_readings = pd.read_csv('data/W512_readings_floorplan1.csv') #original dataset for temp readings 
temperature_readings = temperature_readings.interpolate(method='linear')#fill in null values


weather_readings = pd.read_csv('data/weatherData.csv') #orignal dataset for weather readings

# Converting date and time in temperature_readings to UNIX timestamp for comparision 
temperature_readings['datetime_str'] = temperature_readings['date'] + ' ' + temperature_readings['time']
temperature_readings['datetime'] = temperature_readings['datetime_str'].apply(lambda x: datetime.strptime(x, "%a %b %d %Y %I:%M:%S %p"))
temperature_readings['unix_timestamp'] = temperature_readings['datetime'].apply(lambda x: int(x.timestamp()))

# Converting date and time in weather_readings to UNIX timestamp for comparision 
weather_readings['datetime_str'] = weather_readings['date'] + ' ' + weather_readings['time']
weather_readings['datetime'] = weather_readings['datetime_str'].apply(lambda x: datetime.strptime(x, "%a %b %d %Y %I:%M:%S %p"))
weather_readings['unix_timestamp'] = weather_readings['datetime'].apply(lambda x: int(x.timestamp()))

weather_result_col = [col for col in weather_readings.columns if "result" in col.lower()]

# Merging both data for it to be on the same time
merged_data = pd.merge_asof(
    temperature_readings,  # Left DataFrame
    weather_readings[['unix_timestamp']+weather_result_col],      # Right DataFrame
    on='unix_timestamp',   # Key column
    direction='nearest'    # Match the nearest time
)

datetime_string = merged_data['date'] + " " + merged_data["time"]
merged_data["ISO_formatted_datetime"] = pd.to_datetime(
    datetime_string,
    format="%a %b %d %Y %I:%M:%S %p"
)


# #Columns for lorWan Sensors
temperature_col = [
    col for col in merged_data.columns 
    if "lorawan_readings" in col.lower() and "temperature" in col.lower()
]

humidity_col = [
    col for col in merged_data.columns 
    if "humidity" in col.lower() and "lorawan_readings" in col.lower()
]

co2_col = [
    col for col in merged_data.columns 
    if "co2" in col.lower() and "lorawan_readings" in col.lower()
]

sensors_to_keep = ["Sensor_1", "Sensor_3", "Sensor_6"]
sensors_col = [col for col in merged_data.columns if any(sensor in col for sensor in sensors_to_keep)]

weather_cols_to_keep = ["weather_status","weather_temp","weather_humidity"]
weather_col = [col for col in merged_data.columns if any(weathercol in col for weathercol in weather_cols_to_keep)]

#adding avg temp humid and co2
merged_data['avg_temperature'] = merged_data[temperature_col].mean(axis=1)
merged_data['avg_humidity'] = merged_data[humidity_col].mean(axis=1)
merged_data['avg_co2'] = merged_data[co2_col].mean(axis=1)

avg_col = [
    col for col in merged_data.columns
    if "avg" in col.lower()
]

#Energy(power,energy,current) Data
energy_data = merged_data[["ISO_formatted_datetime"]+ sensors_col]
energy_data.columns = energy_data.columns.str.replace(
    r"Energy_Readings.Sensor_1\.(Current|Energy|Power)", "compressor_\\1", regex=True
).str.replace(
    r"Energy_Readings.Sensor_3\.(Current|Energy|Power)", "fancoil_1_\\1", regex=True
).str.replace(
    r"Energy_Readings.Sensor_6\.(Current|Energy|Power)", "fancoil_2_\\1", regex=True
)

#indoor Data
indoor_data = merged_data[["ISO_formatted_datetime"] + temperature_col + humidity_col + co2_col + avg_col]


#Weather data
weather_data = merged_data[["ISO_formatted_datetime"]+ weather_col]
weather_data.columns = weather_data.columns.str.replace(
    r"result.weather_status", "weather_status", regex=True
).str.replace(
    r"result.weather_temp", "weather_temp", regex=True
).str.replace(
    r"result.weather_humidity", "weather_humidity", regex=True
)


#merging all the needed data
energy_indoor_merged = pd.merge(energy_data, indoor_data, on='ISO_formatted_datetime', how='inner')
final_merged_data = pd.merge(energy_indoor_merged, weather_data, on='ISO_formatted_datetime', how='inner')

final_merged_data['total_energy'] = (
    final_merged_data['compressor_Energy'] +
    final_merged_data['fancoil_1_Energy'] +
    final_merged_data['fancoil_2_Energy']
)

final_merged_data['total_power'] = (
    final_merged_data['compressor_Power'] +
    final_merged_data['fancoil_1_Power'] +
    final_merged_data['fancoil_2_Power']
)

final_merged_data['total_current'] = (
    final_merged_data['compressor_Current'] +
    final_merged_data['fancoil_1_Current'] +
    final_merged_data['fancoil_2_Current']
)
final_merged_data['hour'] = pd.to_datetime(final_merged_data['ISO_formatted_datetime']).dt.hour
final_merged_data['day_of_week'] = pd.to_datetime(final_merged_data['ISO_formatted_datetime']).dt.dayofweek

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def create_target_col(df):
    comfort_condition = (
        (df['avg_co2'] < 1000) &  # CO2 level
        (40 <= df['avg_humidity']) & (df['avg_humidity'] <= 60)
    )
    energy_eff_condtion =(
        (df['total_power'] < 10) & 
        (df['total_current'] < 13)
    )

    comfortable_temp = df.loc[comfort_condition & energy_eff_condtion, 'avg_temperature'].median()

    power_weight = 1 - (df['total_power'] / df['total_power'].max())
    current_weight = 1 - (df['total_current'] / df['total_current'].max())
        
    '''
    Target temperature calculation:
    1. If comfort conditions are met, use the actual temperature
    2. Otherwise, calculate a weighted temperature considering:
        70% of the comfortable mean temperature
        30% of the actual temperature
        Adjusted by power and current consumption weights
    '''
    target = np.where(
        comfort_condition, 
        df['avg_temperature'], 
        0.7 * comfortable_temp + 0.3 * df['avg_temperature'] * (power_weight + current_weight) / 2
    )
    return target
    
     
target = create_target_col(final_merged_data)
final_merged_data['optimal_temp'] = target

final_merged_data['ISO_formatted_datetime']


0      2024-11-15 16:25:07
1      2024-11-15 16:30:06
2      2024-11-15 16:35:06
3      2024-11-15 16:40:05
4      2024-11-15 16:45:06
5      2024-11-15 16:50:06
6      2024-11-15 16:55:06
7      2024-11-15 17:00:05
8      2024-11-15 17:05:06
9      2024-11-15 17:10:06
10     2024-11-15 17:15:06
11     2024-11-15 17:20:06
12     2024-11-15 17:25:06
13     2024-11-15 17:30:05
14     2024-11-15 17:35:06
15     2024-11-15 17:40:06
16     2024-11-15 17:45:06
17     2024-11-15 17:50:06
18     2024-11-15 17:55:06
19     2024-11-15 18:00:06
20     2024-11-15 18:05:06
21     2024-11-15 18:10:06
22     2024-11-15 18:15:06
23     2024-11-15 18:20:05
24     2024-11-15 18:25:06
25     2024-11-15 18:30:06
26     2024-11-15 18:35:06
27     2024-11-15 18:40:06
28     2024-11-15 18:45:06
29     2024-11-15 18:50:06
30     2024-11-15 18:55:06
31     2024-11-15 19:00:06
32     2024-11-15 19:05:06
33     2024-11-15 19:10:06
34     2024-11-15 19:15:06
35     2024-11-15 19:20:06
36     2024-11-15 19:25:06
3

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam

# Relevant columns for input
input_features = [
    'avg_temperature', 'avg_humidity', 'avg_co2',
    'weather_temp', 'weather_humidity', 'total_energy', 
    'total_power', 'total_current', 'hour', 'day_of_week'
]
categorical_features = ['weather_status']

# Preprocessing
X = final_merged_data[input_features + categorical_features]
y = final_merged_data['optimal_temp']

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), input_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)
X = preprocessor.fit_transform(X)

X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print("X shape:", X.shape)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

print("X dtype:", X.dtype)
print("y dtype:", y.dtype)

# Define the LSTM model-building function for KerasTuner
def build_model(hp):
    model = Sequential()
    
    # LSTM layer
    model.add(LSTM(
        units=hp.Int('units_lstm', min_value=32, max_value=128, step=32),
        activation='tanh',
        recurrent_activation='sigmoid',
        kernel_regularizer=l2(hp.Float('l2_lstm', 0.001, 0.1, step=0.001)),
        input_shape=(X_train.shape[1], X_train.shape[2]),
        return_sequences=hp.Boolean('return_sequences')
    ))
    
    model.add(Dropout(rate=hp.Float('dropout_lstm', 0.0, 0.5, step=0.1)))

    
    # Dense layers
    model.add(Dense(
        units=hp.Int('units_dense', min_value=16, max_value=64, step=16),
        activation='relu',
        kernel_regularizer=l2(hp.Float('l2_dense', 0.001, 0.1, step=0.001))
    ))
    
    # Output layer
    model.add(Dense(1, activation='linear'))
    
    # Optimizer
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4]))
    
    # Compile the model
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Initialize KerasTuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',  # Minimize validation loss
    max_trials=100,         # Number of hyperparameter combinations
    executions_per_trial=2,  # Number of runs per combination for stability
    directory='hyperparam_tuning',
    project_name='optimal_temp_lstm'
)

# Train models using hyperparameter search
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True
)

tuner.search(
    X_train, y_train,
    epochs=500,
    validation_split=0.3,
    batch_size=8,
    callbacks=[early_stopping]
)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters for reference
print("Best Hyperparameters:")
print(best_hps.values)

# Rebuild the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Train the model on the full training data
history = best_model.fit(
    X_train, y_train,
    validation_split=0.3,
    epochs=500,
    batch_size=8,
    callbacks=[early_stopping],
    verbose=1
)

# Save the model
best_model.save('models/LSTM_Network_v1.keras')

X shape: (3232, 1, 13)
X_train shape: (2585, 1, 13)
X_test shape: (647, 1, 13)
y_train shape: (2585,)
y_test shape: (647,)
X dtype: float64
y dtype: float64
Reloading Tuner from hyperparam_tuning\optimal_temp_lstm\tuner0.json
Best Hyperparameters:
{'units_lstm': 64, 'l2_lstm': 0.003, 'return_sequences': True, 'dropout_lstm': 0.1, 'units_dense': 64, 'l2_dense': 0.046, 'learning_rate': 0.001}
Epoch 1/500


C:\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 518.8376 - mae: 21.9109 - val_loss: 8.1794 - val_mae: 1.1624
Epoch 2/500
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9.0288 - mae: 1.4806 - val_loss: 5.8094 - val_mae: 0.7528
Epoch 3/500
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7.4072 - mae: 1.2830 - val_loss: 4.7482 - val_mae: 0.5962
Epoch 4/500
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6.3453 - mae: 1.2234 - val_loss: 3.9949 - val_mae: 0.4902
Epoch 5/500
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.3461 - mae: 1.0883 - val_loss: 3.4255 - val_mae: 0.4418
Epoch 6/500
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.8156 - mae: 1.0678 - val_loss: 3.0429 - val_mae: 0.5139
Epoch 7/500
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.2479 - mae: 0.9957 - val_loss: 2.7082 - val_mae: 0.4728
Epoch 8/500
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.7741 - mae: 0.9722 - val_loss: 2.2460 - val_mae: 0.2860
Epoch 9/500
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - lo

In [1]:
import keras
import pandas as pd
import numpy as np
import random
from datetime import datetime
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

# Or ignore specific warning types
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# List of possible weather statuses
weather_statuses = ['Clouds', 'Rain', 'Thunderstorm']

def generate_random_conditions():
    """Generate a dictionary of randomized conditions"""
    return {
        'ISO_formatted_datetime': datetime.now().isoformat(),
        'avg_temperature': round(random.uniform(20, 30), 2),
        'avg_humidity': round(random.uniform(30, 80), 2),
        'avg_co2': round(random.uniform(350, 500), 2),
        'weather_temp': round(random.uniform(22, 33), 2),
        'weather_humidity': round(random.uniform(40, 90), 2),
        'total_current': round(random.uniform(0.5, 1.5), 2),
        'total_energy': round(random.uniform(2000, 5000), 2),
        'total_power': round(random.uniform(5, 15), 2),
        'hour': random.randint(0, 23), 
        'day_of_week': random.randint(0, 6),  
        'weather_status': random.choice(weather_statuses)
    }

# Generate multiple random condition sets
num_scenarios = 5
scenario_results = []

for scenario in range(num_scenarios):
    # Generate a random set of conditions
    current_conditions = generate_random_conditions()
    print(f"\nScenario {scenario + 1} Conditions:")
    for key, value in current_conditions.items():
        print(f"{key}: {value}")
    
    # Create a DataFrame with the current conditions
    df = pd.DataFrame([current_conditions])
    
    # Preprocess
    X_temp = preprocessor.transform(df[input_features + categorical_features])
    X_temp = X_temp.reshape((X_temp.shape[0], 1, X_temp.shape[1]))
    
    # Predict
    prediction = new_model.predict(X_temp)[0][0]
    
    # Print prediction
    print(f"\nPredicted Optimal Temperature: {float(prediction):.2f}°C")
    print("-" * 50)


Scenario 1 Conditions:
ISO_formatted_datetime: 2024-12-06T10:59:39.186682
avg_temperature: 20.76
avg_humidity: 78.06
avg_co2: 351.93
weather_temp: 30.94
weather_humidity: 53.65
total_current: 1.07
total_energy: 4175.22
total_power: 12.59
hour: 20
day_of_week: 6
weather_status: Clouds


NameError: name 'preprocessor' is not defined